In [ ]:
pip install beautifulsoup4

In [ ]:
pip install tqdm

In [ ]:
pip install webdriver_manager

In [ ]:
pip install requests

In [ ]:
pip install webdriver_manager

In [2]:
pip install selenium --upgrade --user

  Using cached selenium-4.8.0-py3-none-any.whl (6.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pyperclip

In [ ]:
pip install pyautogui

In [ ]:
pip tqdm.notebook

In [ ]:
pip install ipywidgets

In [ ]:
pip install openpyxl

In [6]:
from openpyxl import Workbook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import time
import pyperclip
import pyautogui as pag
from bs4 import BeautifulSoup
import requests
from xml.etree import ElementTree
import re
import string
from tqdm.notebook import tqdm
import random

#엑셀파일 생성하기
workbook = Workbook()
sheet = workbook.active

sheet['A1'] = "지역"
sheet['B1'] = "매각 기일"
sheet['C1'] = "사건 번호"
sheet['D1'] = "주소"
sheet['E1'] = "기타 사항"
sheet['F1'] = "자세한 내용"

# 아이디 / 비밀번호
user_id = ''
user_pw = ''


# 옵션
options = webdriver.ChromeOptions()


options.add_argument("lang=ko_KR") # 가짜 플러그인 탑재
# options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36') # user-agent 이름 설정
# 혹은 options.add_argument("--disable-gpu")

driver = webdriver.Chrome('chromedriver', chrome_options=options)

# options = Options()
# options.add_experimental_option("detach", True)
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)

# 지역설정
local = ["경기"]
big_box = []
for where in local:

    url = "https://www.auction1.co.kr/common/login_box.php"

    driver.get(url)
    driver.implicitly_wait(2)

    # ID
    elem_id = driver.find_element(By.NAME, "client_id")
    elem_id.click()
    pyperclip.copy(user_id)
    elem_id.send_keys(Keys.CONTROL, 'v')
    # PASSW
    elem_id = driver.find_element(By.NAME, "pwd_dummy")
    elem_id.click()
    pyperclip.copy(user_pw)
    pag.hotkey('ctrl', 'v')
    pag.press('enter')

    # 종합경매
    elem = driver.find_element(By.XPATH, '/html/body/div[4]/div/div/ul/li[1]/a/span')
    elem.click()

    # 지역 선택
    select = Select(driver.find_element(By.XPATH, '//*[@id="sido"]'))
    select.select_by_visible_text(where)

    # 검색
    driver.find_element(By.XPATH, '//*[@id="fm_aulist"]/table/tbody/tr[9]/td/input[11]').click()
    time.sleep(random.uniform(2,4))
    html3 = driver.page_source

    # 마지막 페이지 확인
    driver.find_element(By.XPATH, '//*[@id="auct_list"]/div[4]/div/a[11]').click()
    last_page = driver.page_source
    last_page = BeautifulSoup(last_page, 'html.parser').select('strong')
    last_page = re.findall(r'\d+', str(last_page))
    last_page = int(last_page[0])
    print("{0}의 마지막 페이지는 {{ {1} }}p 입니다".format(where,last_page))

    # 첫번째 페이지 복귀
    driver.back()

    # 박스/페이지 번호 생성
    box = []
    where_url = 0
    
    
    print(""" 
    ___
    \_/ 
     |._
     |'."-._.-""--.-"-.__.-'/
     |  \                   (
     |   |                  )
     |   |       {0}       /
     |  /                 /
     |.'                 (
     |.-"-.__.-""-.__.-"-.)
     |
     |
     |""".format(where))
    
    for page_num in tqdm(range(101, last_page)):  
        if where == '인천':
            where_url = 28
        if where == '서울':
            where_url = 11 
        if where == '경기':
            where_url = 41
        
        print("-------------현재 {", page_num + 1, "} 페이지 입니다-------------")
        page_num = page_num * 20
        url = "https://www.auction1.co.kr/auction/ca_list.php?total_record=736&search_fm_off=1&search_fm_off=1&lawsup=&lesson=&num1=&num2=&ju_price1=0&ju_price2=0&state=1%2C2%2C17%2C18&b_count1=&b_count2=&s_class=&power_flag=0&bi_price1=0&bi_price2=0&clg_all=&s_class2=&next_biddate1=&next_biddate2=&b_area1=&b_area2=&e_area1=&e_area2=&sido={0}&gugun=0&dong=0&ref_page=&ref_sido=&ref_gugun=&ref_dong=&am_cnt=0&sido_multi=&bunji_key=&bunji1=&bunji2=&address=&address2=&special=0&order=&scale=&sagun_type=&page_code=101010&subcode=&ck_photo=1&favor_mode=&favor_edit=0&from_favor=&from_my_search=&search_mode=1&favor_idx=&queryType=&\
            start={1}".format(where_url, page_num)
        driver.get(url)
        driver.implicitly_wait(2)
        # 페이지 소스 긁기 
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 사건번호 찾기
        a = soup.find_all('td', {'class':'money bold f12'})
        regex = re.compile("\([^<>]+\)")
        line_num = 0
        for i in range(len(a)):
            line_num = line_num + 1
            print("                ", line_num, "번 물건 검사중...")
            # 라인넘버 찾기 { 사건번호 / 라인 넘버 / ? } 순서 
            mc = regex.findall(str(a[i]))
            mc = mc[0].replace('(', '').replace(')', '')
            mc = "".join(mc)
            mc = mc.split(',')
            
            # 새 인터넷창 열기
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
            url = "https://www.auction1.co.kr/auction/ca_view.php?product_id={0}&line_num=1&line_tnum={1}&user_ssid=29393979963c071c9095ab&person_hide=0"\
                    .format(mc[0],mc[1])
            driver.get(url)
            driver.implicitly_wait(2)
            time.sleep(random.uniform(2,4))
            #html 을 문자열로 쪼개기
            html_2 = driver.page_source
            result = BeautifulSoup(html_2, 'html.parser')
            result = result.prettify()
            result_list = result.split('\n')

            #문자열 찾기
            find_string = ['반환','청구권','포기']
            find_string2 = ['임차권', '등기', '말소']
            fs_box = []
            fs_detail_box = []
            count = 0
            st_count = 0
            for i in result_list:
                for j in find_string:
                    if j in i:
                        st_count = st_count + 1
                    if st_count >= len(find_string):                  
                        print("[{0}]".format(j), end=" ")
                        print("[{0}]".format(i))
                        fs_box.append(j)
                        fs_detail_box.append(i)
                        count = count + 1
                st_count = 0
                for j in find_string2:
                    if j in i:
                        st_count = st_count + 1    
                    if st_count >= len(find_string2):               
                        print("[{0}]".format(j), end=" ")
                        print("[{0}]".format(i))
                        fs_box.append(j)
                        fs_detail_box.append(i)
                        count = count + 1
                st_count = 0    
                
            if count > 0 :
                    print("이(가) 포함되어 있습니다.")
    
            if count > 0:
                # 사건번호 찾기
                subject = BeautifulSoup(html_2, 'html.parser').find_all('span', {'class':'num'})
                result = ' '.join(map(str, subject))
                cleanr = re.compile('<.*?>')
                # 사건번호 == cleantext
                cleantext = re.sub(cleanr, '', result)
                print( "사건 번호", ":" , cleantext)
                # 위치 찾기
                location = driver.page_source
                location = BeautifulSoup(location, 'html.parser').find('td', {'class':'addr left'})
                location = ' '.join(map(str, location))
                cleanr = re.compile('<.*?>')
                location = re.sub(cleanr, '', location)
                location = location.replace("\n", "")
                print("주소 : {", location, "}")
                # 매각 기일
                end_day = driver.page_source
                end_day = BeautifulSoup(end_day, 'html.parser').find('span', {'class':'blue bold no'})
                end_day = ' '.join(map(str, end_day))
                cleanr = re.compile('<.*?>')
                end_day = re.sub(cleanr, '', end_day)
                end_day = end_day.replace("\n", "")
                print("매각 기일 {" ,end_day, "}")
                # 엑셀 단위로 데이터 입력하기
                fs_box_result = ' '.join(s for s in fs_box)
                fs_detail_box = ' '.join(s for s in fs_detail_box)
                sheet.append([where, end_day, cleantext, location, fs_box_result, fs_detail_box])
                workbook.save("sample.xlsx")
                # 카운트 리셋
                box.append(end_day)
                box.append(cleantext)
                box.append(location)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        pag.click()
    big_box.append(box) 
    
# driver 종료
driver.quit()


C:\Users\Yune Changhune\AppData\Local\Temp\ipykernel_12512\765083140.py:47: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver', chrome_options=options)


경기의 마지막 페이지는 { 114 }p 입니다
 
    ___
    \_/ 
     |._
     |'."-._.-""--.-"-.__.-'/
     |  \                   (
     |   |                  )
     |   |       경기       /
     |  /                 /
     |.'                 (
     |.-"-.__.-""-.__.-"-.)
     |
     |
     |


  0%|          | 0/13 [00:00<?, ?it/s]

-------------현재 { 102 } 페이지 입니다-------------
                 1 번 물건 검사중...
                 2 번 물건 검사중...
                 3 번 물건 검사중...
                 4 번 물건 검사중...
                 5 번 물건 검사중...
                 6 번 물건 검사중...
                 7 번 물건 검사중...
                 8 번 물건 검사중...
                 9 번 물건 검사중...
                 10 번 물건 검사중...
                 11 번 물건 검사중...
                 12 번 물건 검사중...
[말소] [          ▶임차권자가 배당에서 보증금 전액을 변제받지 못하는 경우 주택임차권등기는 말소되지 않으며, 그 잔액은 매수인이 인수함.]
이(가) 포함되어 있습니다.
사건 번호 : 2021타경3774
주소 : {  경기도 부천시 원종동 291-19 외 1필지, 원종스카이빌 3층 302호           }
매각 기일 { 2023.02.23(木) (10:00) }
                 13 번 물건 검사중...
                 14 번 물건 검사중...
                 15 번 물건 검사중...
                 16 번 물건 검사중...
                 17 번 물건 검사중...
                 18 번 물건 검사중...
                 19 번 물건 검사중...
[말소] [          ☞임차권자가 배당에서 임대차보증금 전액을 배당받지 못하는 경우 주택임차권등기는 말소되지 않으며, 그 잔액은 매수인이 인수함.]
이(가) 포함되어 있습니다.
사건 번호 : 2021타경38176
주소 : {  경기도 부천시 약대동 1

KeyboardInterrupt: 

In [119]:

end_day = driver.page_source
end_day = BeautifulSoup(end_day, 'html.parser').find('span', {'class':'blue bold no'})
end_day
end_day = ' '.join(map(str, end_day))
cleanr = re.compile('<.*?>')
end_day = re.sub(cleanr, '', end_day)
end_day.replace("\n", "")
# end_day.strip("\n")

# regex = re.compile("\([^<>]+\)")
# mc = regex.findall(str(end_day))
# mc = mc[0].replace('(', '').replace(')', '')
# mc = "".join(mc)
# 

' 서울특별시 서대문구 연희동 194-30 외 6필지, 브라운스톤연희 2층 비218호          '

In [15]:

workbook = Workbook()
sheet = workbook.active
sheet['A1'] = "매각 기일"
sheet['B1'] = "사건 번호"
sheet['C1'] = "주소"
sheet['D1'] = "특이 사항"

# row 단위로 데이터 입력하기
sheet.append(['홍길동', 20, '남자'])
sheet.append(['춘향이', 25, '여자', '경기'])
sheet.append(['이몽룡', 30, '남자', ])
sheet.append([end_day, cleantext, location, "특별","반환","임차"])
#시트 추가하기
workbook.create_sheet("성적정보", 1)
sheet2 = workbook['성적정보'] #생성한 시트 접근
sheet2.append(['홍길동', '국어', 100])
sheet2.append(['춘향이', '수학', 90])
sheet2.append(['이몽룡', '사회', 85])
rrr = "썽정섷서서"

#시트 추가하기
workbook.create_sheet("{0}".format(rrr),2)
sheet3 = workbook['{0}'.format(rrr)] #생성한 시트 접근
sheet3.append(['홍길동', '국어', 100])
sheet3.append(['춘향이', '수학', 90])
sheet3.append(['이몽룡', '사회', 85])

# 파일 저장하기
workbook.save("sample.xlsx")

In [5]:
result_list

['<html>',
 ' <head>',
 '  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>',
 '  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>',
 '  <meta content="옥션원은 가장 빠르고 정확하게 경매정보의 표준을 만들어 갑니다. 경매, 공매, NPL, 동산정보를 제공하고 빠르고 다양한 검색조건을 체험하세요." name="description"/>',
 '  <title>',
 '   옥션원-법원경매정보No.1',
 '  </title>',
 '  <link href="/css/common.css?ver=0.02" rel="stylesheet" type="text/css"/>',
 '  <link href="/css/btn_box.css?ver=0.02" rel="stylesheet" type="text/css"/>',
 '  <link href="/img/icon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>',
 '  <script src="/js/jquery-1.7.1.min.js" type="text/javascript">',
 '  </script>',
 '  <script src="/js/jquery.history.js" type="text/javascript">',
 '  </script>',
 '  <script type="text/javascript">',
 '   var user_ssid="42184349863d63357dad68";',
 '  </script>',
 '  <script src="/js/function.js?ver=0.02" type="text/javascript">',
 '  </script>',
 '  <script src="/js/calendar.js?ver=0.01" type="text/javascript">',
 ' 